# Dictionnaire de rimes

Nous allons essayer de construire un dictionnaire de rimes à partir de la ressource [lexique 3.80](http://www.lexique.org/).
À l'aide de la fonction 'recherche par propriété' nous avons pu extraire les mots portant la catégorie 'NOM' du lexique et récupérer pour chacun d'entre les informations suivantes : ortho, phon, lemme, cgram, genre, nombre

Dans le fichier `noms-lexique.org.txt` nous avons (5 premières lignes) :

In [9]:
with open('noms-lexique.org.txt', 'r') as f:
    for i in range(0, 5):
        line = f.readline()
        print(line)

ortho	phon	lemme	cgram	genre	nombre	

a	a	a	NOM	m		

a priori	apRijoRi	a priori	NOM	m		

à-côté	akote	à-côté	NOM	m	s	

à-côtés	akote	à-côté	NOM	m	p	



Notre dictionnaire de rimes devra *in fine* permettre de trouver les rimes possibles d'un mot c-a-d les noms qui ont la même terminaison phonétique.
Pour cela nous devrons conserver deux informations du fichier : ortho et phon  
Commençons pour ça.

 ## Extraction des données

Nous allons parcourir l'intégralité du fichier et ne récupérer que les informations ortho et phon  
C'est un fichier au format `tsv`, les champs sont séparés par une tabulation.

In [11]:
words = []
with open('noms-lexique.org.txt', 'r') as f:
    f.readline() # première ligne
    for line in f:
        fields = line.split('\t')
        ortho = fields[0]
        phon = fields[1]
        words.append((ortho, phon))
words[3:6]

[('à-côtés', 'akote'), ('à-coup', 'aku'), ('à-coups', 'aku')]

On peut synthétiser un peu le code à l'aide des tranches

In [13]:
words = []
with open('noms-lexique.org.txt', 'r') as f:
    f.readline() # première ligne
    for line in f:
        (ortho, phon) = line.split('\t')[0:2]
        words.append((ortho, phon))
words[3:6]

[('à-côtés', 'akote'), ('à-coup', 'aku'), ('à-coups', 'aku')]

Hop, on emballe dans une fonction :

In [ ]:
def extract(file):
    """Extrait du fichier arguement les deux premiers champs
    arg : fichier au format tsv
    return : list de tuples (ortho, phon)
    """
    
    return words

## Constitution de la structure de données

Nous voulons classer les noms de notre liste en fonction des phonèmes de terminaison.  
Et cela selon trois classement différents : par suite de 3 phonèmes, 2 phonèmes et enfin un phonème.  

## Parcours et recherche dans la structure de données

## Exemple

Nous allons utiliser notre dictionnaire de rimes pour modifier la première strophe de Brise Marine de Mallarmé :  

La chair est triste, hélas ! et j'ai lu tous les livres.  
Fuir ! là-bas fuir! Je sens que des oiseaux sont ivres  
D'être parmi l'écume inconnue et les cieux !  
Rien, ni les vieux jardins reflétés par les yeux  
Ne retiendra ce coeur qui dans la mer se trempe  
Ô nuits ! ni la clarté déserte de ma lampe  
Sur le vide papier que la blancheur défend  
Et ni la jeune femme allaitant son enfant.  
Je partirai ! Steamer balançant ta mâture,  
Lève l'ancre pour une exotique nature !  



L'idée est remplacer le dernier mot de chaque vers par un mot pioché au hasard dans notre dictionnaire de rimes.
On prendra d'abord un mot dans la liste des rimes à 3 phonèmes puis une deuxième version à 2 phonèmes.